In [1]:
# This was an attempt to get results at a degraded resolution from data we already have, i.e. generating 200m resolution data from 20m resolution data. Ran into issues with file size and memory. 

In [ ]:
# imports
import numpy as np
import csv
import ee
ee.Initialize()

In [2]:
"""
inputs: file name/location & new resolution in km

1. get bounding box from file - DONE
2. take bounding box and create list of squares in desired resolution - DONE
3. for each new square, find all squares in csv file within the new square - DONE
4. take the average of each column for the squares within the new square - write row to new file - DONE
5. status - DONE

"""

'\ninputs: file name/location & new resolution in km\n\n1. get bounding box from file - DONE\n2. take bounding box and create list of squares in desired resolution - DONE\n3. for each new square, find all squares in csv file within the new square - DONE\n4. take the average of each column for the squares within the new square - write row to new file - DONE\n5. status - DONE\n\n'

In [3]:
file = "results_bigregion_bottom/compiled.csv"
new_res = .2 # 200 m = .2 km, 20 m = .02 km

In [4]:
#ex = np.genfromtxt(file, delimiter=',', skip_header=1, skip_footer=0, usecols = (0, 1, 2, 3, 7))
ex = np.genfromtxt(file, delimiter=',', skip_header=1, skip_footer=0)

KeyboardInterrupt: 

In [ ]:
#find bounding box
min_lon = np.amin(ex[:,0]);
min_lat = np.amin(ex[:,1]);
max_lon = np.amax(ex[:,2]);
max_lat = np.amax(ex[:,3]);
print(min_lon)
print(min_lat)
print(max_lon)
print(max_lat)

In [ ]:
bounding_box = ee.Geometry.Polygon([[[min_lon, min_lat], [min_lon, max_lat], [max_lon, max_lat], [max_lon, min_lat]]])

In [ ]:
def create_segments(geometry, size):
    # grab the top left coordinate of the bounding box
    coords = ee.List(geometry.coordinates().get(0)).slice(0, -1)
    top = ee.Number(ee.List(coords.get(2)).get(1))
    left = ee.Number(ee.List(coords.get(0)).get(0))
    top_left_point = ee.Geometry.Point([left, top])
    
    # calculate square pixel width and height
    width = int(ee.Geometry.Point(coords.get(0)).distance(ee.Geometry.Point(coords.get(1))).divide(1000 * size).getInfo()) # how many squares can fit in the width of the bounding box
    height = int(ee.Geometry.Point(coords.get(1)).distance(ee.Geometry.Point(coords.get(2))).divide(1000 * size).getInfo()) # how many squares can fit in the height of the bounding box
    
    # create a point buffer and use it to find the km distance to lat/lon degree conversion
    buff = top_left_point.buffer(size*1000, 0.1) # create the buffer with a max % error of 0.1
    buff_list = ee.List(buff.coordinates().get(0)) 
    buff_length = buff_list.length()
    right_pt = ee.List(buff_list.get(buff_length.multiply(0.75).int().subtract(1)))
    bottom_pt = ee.List(buff_list.get(buff_length.multiply(0.5).int().subtract(1)))
    new_lat = ee.Number(right_pt.get(0)) # given distance east of the top left point
    new_lon = ee.Number(bottom_pt.get(1)) # given distance south of the top left point
    
    diff_lon = top.subtract(new_lon) # given size converted to degrees
    diff_lat = new_lat.subtract(left)
    
    # build the list of squares
    segments = []
    
    for y in range(height + 1): # +1 to guarantee we will cover the whole region (squares may extend slightly past the bounding box)
        left = ee.Number(ee.List(coords.get(0)).get(0))
        for x in range(width + 1):
            new_lat = left.add(diff_lat)
            new_lon = top.subtract(diff_lon)
            
            square = ee.Geometry.Polygon(
                [[[left, new_lon],
                  [new_lat, new_lon],
                  [new_lat, top],
                  [left, top]]])
            
            segments.append(square)
            
            left = new_lat
        top = new_lon
        
    return segments

In [8]:
squares = create_segments(bounding_box, new_res)

In [9]:
print(len(squares))

240


In [10]:
print(np.size(ex[:,1]))

22950


In [11]:
def create_results(feature):
    coords = ee.List(feature.geometry().coordinates().get(0))
    lon_min = ee.List(coords.get(0)).get(0)
    lon_max = ee.List(coords.get(1)).get(0)
    lat_min = ee.List(coords.get(0)).get(1)
    lat_max = ee.List(coords.get(2)).get(1)
    row = ee.Array([lon_min, 
                   lat_min, 
                   lon_max,
                   lat_max])
    new_feature = ee.Feature(None, {'info': row})
    return new_feature

# Calculate values for 250m x 250m squares
regions = create_segments(bounding_box, new_res)
segments = ee.FeatureCollection(regions)

# Create above array for each segment, and transform into format that can be written to a CSV file
data_set = segments.map(create_results)
data_set2 = data_set.aggregate_array('info')
data_set3 = data_set2.getInfo() # <- slow

In [12]:
print(len(data_set3))

720


In [12]:
# Create CSV and add header & data
with open('squares_mine.csv', 'w') as f:
    writer = csv.writer(f)
    #writer.writerow(header_list)
    writer.writerows(data_set3)

In [13]:
with open('squares_mine.csv', 'r') as r, \
        open('200m_mine.csv', 'w', newline='') as w:
    # Create a csv.reader object from the input file object
    csv_reader = csv.reader(r)
    # Create a csv.writer object from the output file object
    csv_writer = csv.writer(w)
    # Add passing rows to new file
    # header_list = ['Mininum Longitude', 'Minimum Latitude', 'Maximum Longitude', 'Maximum Latitude', 'Percent Vegetation Loss', \
    #                'Percent Bare Initial','Percent Significant VH Values', 'Average NIR/G', 'Average SWIR1/B', 'NASADEM Elevation', \
    #                'GEDI Elevation','GEDI-SRTM Elevation','GEDI Quality Flag', 'B5', 'B6', 'NDMI', 'Center Lon', \
    #                'Center Lat','Elevation Score', 'Band Variation Score']
    header_list = ['Mininum Longitude', 'Minimum Latitude', 'Maximum Longitude', 'Maximum Latitude', 'Number of small squares', \
                   'Percent Vegetation Loss', 'Percent Bare Initial','Percent Significant VH Values', 'Average NIR/G', 'Average SWIR1/B', \
                   'Elevation Score', 'Band Variation Score']
    csv_writer.writerow(header_list)
    for row in csv_reader:
        min_lon = row[0]
        min_lat = row[1]
        max_lon = row[2]
        max_lat = row[3]
        squares = []
        for row2 in ex:
            if float(row2[0]) >= float(min_lon) and float(row2[1]) >= float(min_lat) \
                and float(row2[2]) <= float(max_lon) and float(row2[3]) <= float(max_lat):
                squares.append([row2])
                
        #print(squares)
        percent_veg_total = 0
        percent_bare_total = 0
        percent_sar_total = 0
        gndvi_total = 0
        swirb_total = 0
        elev_score_total = 0
        band_score_total = 0
        num = 0
        for x in squares:
            percent_veg_total = percent_veg_total + x[0][4]
            percent_bare_total = percent_bare_total + x[0][5]
            percent_sar_total = percent_sar_total + x[0][6]
            gndvi_total = gndvi_total + x[0][7]
            swirb_total = swirb_total + x[0][8]
            elev_score_total = elev_score_total + x[0][18] #17 without NDMI
            band_score_total = band_score_total + x[0][19] #18 without NDMI
            num = num+1
        if num >0:
            avg_veg = percent_veg_total/num
            avg_bare = percent_bare_total/num
            avg_sar = percent_sar_total/num
            avg_gndvi = gndvi_total/num
            avg_swirb = swirb_total/num
            avg_elev_score = elev_score_total/num
            avg_band_score = band_score_total/num
        else: 
            avg_veg = -999
            avg_bare = -999
            avg_sar = -999
            avg_gndvi = -999
            avg_swirb = -999
            avg_elev_score = -999
            avg_band_score = -999
        new_row = [min_lon, min_lat, max_lon, max_lat, num, avg_veg, avg_bare, avg_sar, avg_gndvi, avg_swirb, avg_elev_score, avg_band_score]
        csv_writer.writerow(new_row)

In [14]:
# For a given CSV file of region coordinates and test values, calculate and document whether this region may be a mining location
# All results will be saved in "[file_name]_status.csv"
# Passing region results only will be saved in "[file_name]_status_passing.csv"

# Open the input file in read mode and output file in write mode
with open('200m_mine.csv', 'r') as read_obj, \
        open('200m_mine_status.csv', 'w', newline='') as write_obj:
    # Create a csv.reader object from the input file object
    csv_reader = csv.reader(read_obj)
    next(csv_reader) # skip header
    # Create a csv.writer object from the output file object
    csv_writer = csv.writer(write_obj)
    # Add header to output file, with status column
    header_list = ['Mininum Longitude', 'Minimum Latitude', 'Maximum Longitude', 'Maximum Latitude', 'Number of small squares', \
                   'Percent Vegetation Loss', 'Percent Bare Initial','Percent Significant VH Values', 'Average NIR/G', 'Average SWIR1/B', \
                   'Elevation Score', 'Band Variation Score']
    csv_writer.writerow(header_list)
    # Read each row of the input csv file as list
    for row in csv_reader:
        """
        Calculate Status and append to the end of the row/list
        Passing Criteria:
        If Vegetation Loss < 20% and Bare Earth > 20%: 
            SAR VH > 25% and NIR/G <= 0.3 and SWIR1/B < 0.65 and Elevation Score >= 5 and B5/B6 Score >= 4
        Else: 
            Vegetation Loss > 20% and SAR VH > 25% and NIR/G <= 0.3 and SWIR1/B < 0.65
        """
        vegetation_loss = float(row[5])
        percent_bare = float(row[6])
        sar_vh = float(row[7])
        nir_g = float(row[8])
        swir1_b = float(row[9])
        elevation_score = float(row[10])
        b5_b6_score = float(row[11])
        
        if vegetation_loss == -999:
            status = 0
        elif vegetation_loss < 20 and percent_bare > 20:
            status = sar_vh > 25 and nir_g <= 0.3 and swir1_b < 0.65 and elevation_score >= 5 and b5_b6_score >= 4
        else: 
            status = vegetation_loss > 20 and sar_vh > 25 and nir_g <= 0.3 and swir1_b < 0.65

        if status:
            row.append('Pass')
        else: 
            row.append('Fail')
        # Add the updated row / list to the output file
        csv_writer.writerow(row)

# Create a file of only passing statuses
with open('200m_mine_status.csv', 'r') as r, \
        open('200m_mine_status_passing.csv', 'w', newline='') as w:
    # Create a csv.reader object from the input file object
    csv_reader = csv.reader(r)
    # Create a csv.writer object from the output file object
    csv_writer = csv.writer(w)
    # Add header to output file, with status column
    csv_writer.writerow(header_list)
    # Skip Header in input file
    header = next(csv_reader)
    if header != None:
        # Add passing rows to new file
        for row in csv_reader:
            if row[12] == "Pass":
                csv_writer.writerow(row)
                
# Convert to a Feature Collection
# Create a list of Geometries for areas that passed as possible mines
with open('200m_mine_status_passing.csv', 'r') as r:
    csv_reader = csv.reader(r)
    # Skip Header in input file
    header = next(csv_reader)
    region_list = []
    if header != None:
        # Convert each region to a Geometry and add to a list
        for row in csv_reader:
            region = ee.Geometry.Polygon([[[float(row[0]), float(row[3])],
                      [float(row[0]), float(row[1])],
                      [float(row[2]), float(row[1])],
                      [float(row[2]), float(row[3])]]])
            region_list.append(region)

# Wrap geometry list in a Feature Collection
fc = ee.FeatureCollection(region_list)

# Export the Feature Collection to Google Earth Engine (GEE)
task = ee.batch.Export.table.toAsset(**{
  'collection': fc,
  'description':'compiled_results',
  'assetId': 'users/EmilyNason/trying_something_again', # change to your GEE Asset path and a unique name (will not overwrite already existing assets, so old names cannot be reused)
});

task.start()